In [7]:
#1. get xmlboxes
#2. get xmlboxes img region from bkg
#3. put the img region to new image
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
generated_image_dir = '/data/darknet/python/' + time_str + '_images_replaced_ordinary_obj_with_bkg/'
time_str = '20190603'
xml_dir = '/data/darknet/python/' + time_str + '_xml/'
image_dir = '/data/darknet/python/' + time_str + '_image/'

os.system('mkdir -p ' + xml_dir)
os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
background_dir = '/data/darknet/python/' + time_str + '_background/'

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
SEED_BKG_DIR = background_dir
GENE_IMG_DIR = generated_image_dir


In [8]:
def past_to_background_from_image_file(file, bboxes, background_img_array, save_name, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    '''if(img.shape != background_img_array.shape):
        print('shape not match')
        return'''
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S.%f")
    cv2.imwrite(GENE_IMG_DIR + time_str + '_' + save_name.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml_name):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)
seed_bkg_names.sort()
seed_xml_names.sort()

for xml_name in seed_xml_names:
    print(xml_name)       
    '''if(not ((int(xml_name[:-4]) >= 460500) and int(xml_name[:-4]) < 461000)):
        continue'''
    background_img_data = cv2.imread(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    bboxes = get_bboxes_from_etree(tree)
    past_to_background_from_image_file(SEED_BKG_DIR + seed_bkg_names[0], bboxes, background_img_data, 
                                       SEED_IMG_DIR + xml_name[:-3] + 'jpg')

00006005300.xml
00006005301.xml
00006005302.xml
00006005303.xml
00006005304.xml
00006005305.xml
00006005306.xml
00006005307.xml
00006005308.xml
00006005309.xml
00006005310.xml
00006005311.xml
00006005312.xml
00006005313.xml
00006005314.xml
00006005315.xml
00006005316.xml
00006005317.xml
00006005318.xml
00006005319.xml
00006005320.xml
00006005321.xml
00006005322.xml
00006005323.xml
00006005324.xml
00006005325.xml
00006005326.xml
00006005327.xml
00006005328.xml
00006005329.xml
00006005330.xml
00006005331.xml
00006005332.xml
00006005333.xml
00006005334.xml
00006005335.xml
00006005336.xml
00006005337.xml
00006005338.xml
00006005339.xml
00006005340.xml
00006005341.xml
00006005342.xml
00006005343.xml
00006005344.xml
00006005345.xml
00006005346.xml
00006005347.xml
00006005348.xml
00006005349.xml
00006005350.xml
00006005351.xml
00006005352.xml
00006005353.xml
00006005354.xml
00006005355.xml
00006005356.xml
00006005357.xml
00006005358.xml
00006005359.xml
00006005360.xml
00006005361.xml
00006005

00006005814.xml
00006005815.xml
00006005816.xml
00006005817.xml
00006005818.xml
00006005819.xml
00006005820.xml
00006005821.xml
00006005822.xml
00006005823.xml
00006005824.xml
00006005825.xml
00006005826.xml
00006005827.xml
00006005828.xml
00006005829.xml
00006005830.xml
00006005831.xml
00006005832.xml
00006005833.xml
00006005834.xml
00006005835.xml
00006005836.xml
00006005837.xml
00006005838.xml
00006005839.xml
00006005840.xml
00006005841.xml
00006005842.xml
00006005843.xml
00006005844.xml
00006005845.xml
00006005846.xml
00006005847.xml
00006005848.xml
00006005849.xml
00006005850.xml
00006005851.xml
00006005852.xml
00006005853.xml
00006005854.xml
00006005855.xml
00006005856.xml
00006005857.xml
00006005858.xml
00006005859.xml
00006005860.xml
00006005861.xml
00006005862.xml
00006005863.xml
00006005864.xml
00006005865.xml
00006005866.xml
00006005867.xml
00006005868.xml
00006005869.xml
00006005870.xml
00006005871.xml
00006005872.xml
00006005873.xml
00006005874.xml
00006005875.xml
00006005

00006006631.xml
00006006632.xml
00006006633.xml
00006006634.xml
00006006635.xml
00006006636.xml
00006006637.xml
00006006638.xml
00006006639.xml
00006006640.xml
00006006641.xml
00006006642.xml
00006006643.xml
00006006644.xml
00006006645.xml
00006006646.xml
00006006647.xml
00006006648.xml
00006006649.xml
00006006650.xml
00006006651.xml
00006006652.xml
00006006653.xml
00006006654.xml
00006006655.xml
00006006656.xml
00006006657.xml
00006006658.xml
00006006659.xml
00006006660.xml
00006006661.xml
00006006662.xml
00006006663.xml
00006006664.xml
00006006665.xml
00006006666.xml
00006006667.xml
00006006668.xml
00006006669.xml
00006006670.xml
00006006671.xml
00006006672.xml
00006006673.xml
00006006674.xml
00006006675.xml
00006006676.xml
00006006677.xml
00006006678.xml
00006006679.xml
00006006680.xml
00006006681.xml
00006006682.xml
00006006683.xml
00006006684.xml
00006006685.xml
00006006686.xml
00006006687.xml
00006006688.xml
00006006689.xml
00006006690.xml
00006006691.xml
00006006692.xml
00006006